In [2]:
import numpy as np
import math

In [5]:
import sys
sys.path.append("/Users/lality/projects/personal/ML_Algo/")

In [6]:
from ml.tools import *

In [50]:
from sklearn import datasets

In [51]:
def accuracy_score(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred, axis=0) / len(y_true)
    return accuracy

In [14]:
def shuffle_data(X, y, seed=None):
    if seed:
        np.random.seed(seed)
    idx = np.arange(X.shape[0])
    np.random.shuffle(idx)
    return X[idx], y[idx]

In [17]:
def train_test_split(X, y, test_size=0.2, shuffle=True, seed=None):
    if shuffle:
        X, y = shuffle_data(X, y, seed)
    split_i = len(y) - int(len(y) // (1 / test_size))
    X_train, X_test = X[: split_i], X[split_i:]
    y_train, y_test = y[: split_i], y[split_i:]
    
    return X_train, X_test, y_train, y_test

In [36]:
class DecisionStump:
    def __init__(self):
        self.polarity = 1
        self.feature_index = None
        self.threshold = None
        self.alpha = None

In [48]:
class Adaboost:
    def __init__(self, n_clf=5):
        self.n_clf = n_clf
    
    def fit(self, X, y):
        n_samples, n_features = np.shape(X)
        w = np.full(n_samples, (1 / n_samples))
        
        self.clfs = []
        for _ in range(self.n_clf):
            clf = DecisionStump()
            min_error = float('inf')

            for feature_i in range(n_features):
                feature_values = np.expand_dims(X[: feature_i], axis=1)
                unique_values = np.unique(feature_values)
                
                for threshold in unique_values:
                    p = 1
                    prediction = np.ones(np.shape(y))
                    prediction[X[:, feature_i] < threshold] = -1
                    error = sum(w[y != prediction])
                    
                    if error > 0.5:
                        error = 1 - error
                        p = -1
                    
                    if error < min_error:
                        clf.polarity = p
                        clf.threshold = threshold
                        clf.feature_index = feature_i
                        min_error = error
            clf.alpha = 0.5 * math.log((1.0 - min_error) / (min_error + 1e-10))
            predictions = np.ones(np.shape(y))
            negetive_idx = (clf.polarity * X[:, clf.feature_index] < clf.polarity * clf.threshold)
            predictions[negetive_idx] = -1

            w *=  np.exp(-clf.alpha * y * predictions)
            w /=  np.sum(w)

            self.clfs.append(clf)
        
    def predict(self, X):
        n_samples = np.shape(X)[0]
        y_pred = np.zeros((n_samples, 1))
        
        for clf in self.clfs:
            predictions = np.ones(np.shape(y_pred))
            negetive_idx = (clf.polarity * X[:, clf.feature_index] < clf.polarity * clf.threshold)
            
            predictions[negetive_idx] = -1
            y_pred += clf.alpha * predictions
        y_pred = np.sign(y_pred).flatten()
        
        return y_pred
                        

In [53]:
def main():
    data = datasets.load_digits()
    X = data.data
    y = data.target

    digit1 = 1
    digit2 = 8
    idx = np.append(np.where(y == digit1)[0], np.where(y == digit2)[0])
    y = data.target[idx]
    # Change labels to {-1, 1}
    y[y == digit1] = -1
    y[y == digit2] = 1
    X = data.data[idx]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

    # Adaboost classification with 5 weak classifiers
    clf = Adaboost(n_clf=5)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    print ("Accuracy:", accuracy)

    # Reduce dimensions to 2d using pca and plot the results
#     Plot().plot_in_2d(X_test, y_pred, title="Adaboost", accuracy=accuracy)


if __name__ == "__main__":
    main()

Accuracy: 0.8876404494382022


In [23]:
np.full(5, (1 / 5))

array([0.2, 0.2, 0.2, 0.2, 0.2])